In [127]:
import email
import imaplib
from datetime import datetime
from io import BytesIO

import pandas as pd
from bs4 import BeautifulSoup

In [128]:
prev = 0


def pretty(label: str, completed: float, total: int, length: int = 30) -> None:
    global prev
    print(
        " " * (prev * 2)
        + f"\r{label} ["
        + "=" * int(completed / total * length)
        + "-" * int((total - completed) / total * length)
        + f"] {int(completed / total * 100)}%",
        end="",
    )
    prev = length


def prettyPrintReplace(label: str) -> None:
    global prev
    label = str(label)
    print(" " * (prev * 2) + f"\r{label}", end="")
    prev = len(label)

In [129]:
# Login credentials
IMAP_HOST = "imap.gmail.com"
EMAIL_USER = "ebadansari414@gmail.com"
EMAIL_PASS = "ghgy lcsp obph sczi"

In [130]:
subject_to_search = "Odoo ERP - Trying to reach you"
subject_to_search = "Fwd: UPGRADE MANDI - Hyperpure"

In [131]:
# Connect and login
mail = imaplib.IMAP4_SSL(IMAP_HOST)
mail.login(EMAIL_USER, EMAIL_PASS)
mail.select("inbox")

# Search emails by subject
status, messages = mail.search(None, f'(SUBJECT "{subject_to_search}")')

In [132]:
grnPrDF = pd.DataFrame({"GRN": [], "PO Amount": []})

if status == "OK":
    for num in messages[0].split():
        typ, data = mail.fetch(num, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])

        processedDOMTree = ""

        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition") or "")

            if "attachment" in content_disposition:
                continue
            # if content_type == "text/plain":
            # email_text += part.get_payload(decode=True).decode(errors="ignore")

            # if you also want HTML body
            elif content_type == "text/html":
                rawHTMLText = part.get_payload(decode=True).decode()

                processedDOMTree = BeautifulSoup(rawHTMLText, "html.parser")

                amountDate = datetime.strptime(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[3]
                    .find_all("td")[1]
                    .find("span")
                    .text,
                    "%d %b %Y",
                )

                amount = float(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[4]
                    .find_all("td")[1]
                    .find("span")
                    .text
                )

                isGrnAmount = (
                    "UPGRADE MANDI - Hyperpure GRN against PO Number" in msg["subject"]
                )
                isTotalAmount = "UPGRADE MANDI - Hyperpure PO Number" in msg["subject"]

                prettyPrintReplace(len(grnPrDF))

                if amountDate.strftime("%d-%m-%Y") not in grnPrDF.index:
                    grnPrDF.loc[amountDate.strftime("%d-%m-%Y")] = [0.0, 0.0]
                if isGrnAmount:
                    grnPrDF.loc[amountDate.strftime("%d-%m-%Y"), "GRN"] += amount
                if isTotalAmount:
                    grnPrDF.loc[amountDate.strftime("%d-%m-%Y"), "PO Amount"] += amount

5                                      

In [ ]:
grnPrDF["PR"] = grnPrDF["PO Amount"] - grnPrDF["GRN"]

In [135]:
grnPrDF = grnPrDF[["GRN", "PR", "PO Amount"]]

In [136]:
mail.logout()

('BYE', [b'LOGOUT Requested'])

In [137]:
grnPrDF.index.name = "Date"

In [138]:
grnPrDF.to_csv("GRN PR (Blinkit).csv")